In [1]:
import pandas as pd
from datetime import datetime

# Load and prepare price data
price_df = pd.read_csv(r"C:\Users\satya\Downloads\QR - JP Morgan\Nat_Gas.csv")
price_df['Dates'] = pd.to_datetime(price_df['Dates'], format='%m/%d/%y')
price_df['Prices'] = price_df['Prices'].astype(float)
price_df.set_index('Dates', inplace=True)

def price_storage_contract(injection_dates, withdrawal_dates, injection_rate, withdrawal_rate,
                           max_volume, storage_cost_per_month):
    """
    Calculate the value of a natural gas storage contract.

    Parameters:
    - injection_dates: list of 'YYYY-MM-DD' strings
    - withdrawal_dates: list of 'YYYY-MM-DD' strings
    - injection_rate: volume injected per date
    - withdrawal_rate: volume withdrawn per date
    - max_volume: maximum storage capacity
    - storage_cost_per_month: fixed cost per month of storage

    Returns:
    - Net contract value (float)
    """
    # Convert date strings to datetime
    injection_dates = [pd.to_datetime(date) for date in injection_dates]
    withdrawal_dates = [pd.to_datetime(date) for date in withdrawal_dates]

    # Initialize variables
    total_injected = 0
    total_withdrawn = 0
    cash_flows = []

    # Injection phase
    for date in injection_dates:
        price = price_df.loc[price_df.index == date, 'Prices'].values
        if price.size == 0:
            continue  # skip if date not found
        if total_injected + injection_rate <= max_volume:
            total_injected += injection_rate
            cash_flows.append(-price[0] * injection_rate)  # cost to buy gas

    # Withdrawal phase
    for date in withdrawal_dates:
        price = price_df.loc[price_df.index == date, 'Prices'].values
        if price.size == 0:
            continue
        if total_withdrawn + withdrawal_rate <= total_injected:
            total_withdrawn += withdrawal_rate
            cash_flows.append(price[0] * withdrawal_rate)  # revenue from selling gas

    # Storage cost calculation
    active_months = pd.date_range(start=min(injection_dates), end=max(withdrawal_dates), freq='M')
    total_storage_cost = len(active_months) * storage_cost_per_month
    cash_flows.append(-total_storage_cost)

    # Net contract value
    return round(sum(cash_flows), 2)

In [2]:
# Example inputs
injection_dates = ['2023-06-30', '2023-07-31', '2023-08-31']
withdrawal_dates = ['2023-11-30', '2023-12-31']
injection_rate = 1000  # units
withdrawal_rate = 1500  # units
max_volume = 5000  # units
storage_cost_per_month = 2000  # USD

# Run the model
contract_value = price_storage_contract(injection_dates, withdrawal_dates,
                                        injection_rate, withdrawal_rate,
                                        max_volume, storage_cost_per_month)

print("Estimated Contract Value:", contract_value)

Estimated Contract Value: -9900.0


C:\Users\satya\AppData\Local\Temp\ipykernel_9408\3752719088.py:54: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  active_months = pd.date_range(start=min(injection_dates), end=max(withdrawal_dates), freq='M')


In [3]:
# Example 2: Longer injection period, staggered withdrawals
injection_dates = ['2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31']
withdrawal_dates = ['2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29']
injection_rate = 1200  # units per injection
withdrawal_rate = 1000  # units per withdrawal
max_volume = 6000  # total storage capacity
storage_cost_per_month = 2500  # USD per month

# Run the model
contract_value = price_storage_contract(injection_dates, withdrawal_dates,
                                        injection_rate, withdrawal_rate,
                                        max_volume, storage_cost_per_month)

print("Estimated Contract Value (Example 2):", contract_value)

Estimated Contract Value (Example 2): -28520.0


C:\Users\satya\AppData\Local\Temp\ipykernel_9408\3752719088.py:54: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  active_months = pd.date_range(start=min(injection_dates), end=max(withdrawal_dates), freq='M')
